In [3]:
import datetime
import pandas as pd
import numpy as np
import panel as pn
import plotly.graph_objects as go
from ipywidgets import widgets, Dropdown, VBox, HBox
#from IPython.display import display

pn.extension('plotly')

# Set credentials
#chart_studio.tools.set_credentials_file(username='seyoumyg', api_key='jbpQ2OXKFUBkCusIuicQ')


# Read and merge datasets
df1 = pd.read_csv("Daily_179.csv")
df2 = pd.read_csv("Daily_192.csv")
df3 = pd.read_csv("Daily_204.csv")
df4 = pd.read_csv("Daily_237.csv")
df5 = pd.read_csv("Daily_240.csv")

df = pd.concat([df1, df2, df3, df4, df5])

# Filter and reshape
target_cols = ['Stn Name', 'Date', 'Precip (in)', 'Max Air Temp (F)', 'Min Air Temp (F)', 
               'Avg Air Temp (F)', 'Avg Rel Hum (%)', 'Avg Wind Speed (mph)']
dff = df[target_cols]
df_long = pd.melt(
    dff,
    id_vars=['Stn Name', 'Date'],
    value_vars=[
        'Precip (in)', 'Max Air Temp (F)', 'Min Air Temp (F)',
        'Avg Air Temp (F)', 'Avg Rel Hum (%)', 'Avg Wind Speed (mph)'
    ]
).dropna()

# Widgets
station_selector = pn.widgets.Select(
    name="Lake",
    options=sorted(df_long['Stn Name'].unique().tolist()),
    value='Diamond Valley Lake'
)

variable_selector = pn.widgets.Select(
    name="Weather Variable",
    options=sorted(df_long['variable'].unique().tolist()),
    value='Precip (in)'
)

# Plot function
@pn.depends(station_selector, variable_selector)
def plot_graph(station, variable):
    filtered = df_long[(df_long['Stn Name'] == station) & (df_long['variable'] == variable)]
    fig = go.Figure()
    fig.add_scatter(x=filtered['Date'], y=filtered['value'], mode='lines')
    fig.update_layout(
        title=f"{variable} from {station}",
        yaxis_title=variable,
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=7, label="1 week", step="day", stepmode="backward"),
                    dict(count=1, label="1 m", step="month", stepmode="backward"),
                    dict(count=6, label="6 m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1 year", step="year", stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(visible=True),
            type="date"          
        ),
        height=500, width=1800
    )
    return fig

instructions = pn.pane.Markdown("""
### 📘 Dashboard information and data source
-This dashboard is a work in progress to display weather data from weather stations near Silverwood Lake (Station #192), Diamond Valley Lake (#179), Lake Skinner (#237), 
  Lake Mathews and Perris (#240), and Castaic Lake (#204). 
-To update the dashboard, data is retrieved daily from the California Irrigation Management Information System (CIMIS): https://cimis.water.ca.gov/Default.aspx

 ### 📘 How to Use the Dashboard     
-Use the **Lake** and **Weather parameter** dropdowns to select data.
- The **interactive plot** shows values over time.
- Use range selection buttons on the top-left side of the graph to select the most recent 1 week, 1 month, 6 months, 1 year, and year-to-date data. 
- Use the **click-and-drag range selector below the x-axis** to zoom into a specific date range.
- Double-click on the x-axis to reset the zoom.
""", styles={'font-size': '18px', 'margin-top': '20px'})
# Footer with disclaimer
footer = pn.pane.Markdown("""
---
**Disclaimer:** This dashboard is for the internal use of the Metropolitan Water District of Southern California (MWDSC). 

*Created as part of the experimental project for data visualization*
""",styles={'font-size': '18px', 'margin-top': '20px'})

# Contact information
contact_info = pn.pane.Markdown("""
---
**For questions or feedback, please contact:**  
Seyoum Gebremariam (sgebremariam@madh2o.com)

""", styles={'font-size': '18px', 'margin-top': '20px'})
# Layout
dashboard = pn.Column(
    "# 📊 Interactive Weather Dashboard",
    instructions,
    footer,
    contact_info,
    pn.Row(station_selector, variable_selector),
    plot_graph
)
#py.plot(fig, filename='Shared Plotly Chart', auto_open=True)
dashboard

Column
    [0] Markdown(str)
    [1] Markdown(str, styles={'font-size': '18px', ...})
    [2] Markdown(str, styles={'font-size': '18px', ...})
    [3] Markdown(str, styles={'font-size': '18px', ...})
    [4] Row
        [0] Select(name='Lake', options=['Castaic Lake', ...], value='Diamond Valley Lake')
        [1] Select(name='Weather Variable', options=['Avg Air Temp (F)', ...], value='Precip (in)')
    [5] ParamFunction(function, _pane=Plotly, defer_load=False)